In [1]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH02-Runnable")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH02-Runnable


# RunnableLambda

- 사용자 정의 함수 실행할 수 있도록 기능 제공 
- 가령 데이터 전처리, 계산, 외부API 호출 등의 작업을 수행하는 함수 정의 및 실행 

(주의사항) 
- 해당 함수 자체가 받을 수 있는 인자는 1개 !!!!!
- 여러 개의 인자 입력 받아야 하는 경우 래퍼 작성 

In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

In [4]:
# 텍스트의 길이를 반환하는 함수
def length_function(text):  
    return len(text)

# 두 텍스트의 길이를 곱하는 함수
def _multiple_length_function(text1, text2):  
    return len(text1) * len(text2)

# 2개 인자를 받는 함수로 연결하는 wrapper 함수
def multiple_length_function(  
    _dict,
): 
    return _multiple_length_function(_dict["text1"], _dict["text2"])

In [6]:
# 프롬프트
prompt = PromptTemplate.from_template("{a} + {b}는 무엇입니까?")

# 모델
model = ChatOpenAI()

# 체인 구성
chain = (
    {
        "a": itemgetter("input_1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("input_1"), "text2": itemgetter("input_2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
    | StrOutputParser()    
)

In [7]:
chain.invoke({"input_1": "hello", "input_2": "skala"})

'5 + 25 = 30입니다.'

# RunnableConfig

In [11]:
from langchain_core.runnables import RunnableConfig
from langchain.callbacks import get_openai_callback
import json

In [12]:
def parse_or_fix(text: str, config: RunnableConfig):
    # 텍스트를 수정하는 프롬프트 템플릿 생성
    fixing_chain = (
        PromptTemplate.from_template(
            "Fix the following text:\n\ntext\n{input}\n\nError: {error}"
            "Don't narrate, just respond with the fixed data."
        )
        | ChatOpenAI()
        | StrOutputParser()
    )
    
    # 최대 3번 시도
    for _ in range(3):
        try:
            # JSON 형식으로 텍스트 파싱
            return json.loads(text)
        except Exception as e:
            # 파싱 중 오류가 발생하면 수정 체인을 호출하여 텍스트 수정
            text = fixing_chain.invoke({"input": text, "error": e}, config)
            print(f"config: {config}")
    
    # 파싱에 실패한 경우 
    return "Failed to parse"

In [15]:
with get_openai_callback() as cb:
    output = RunnableLambda(parse_or_fix).invoke(
        input="{foo:: bar}",
        config={"tags": ["my-tag"], "callbacks": [cb]}, 
    )
    
    print(f"\n\n수정한결과:\n{output}")

config: {'tags': ['my-tag'], 'metadata': {}, 'callbacks': <langchain_core.callbacks.manager.CallbackManager object at 0x124e27a50>, 'recursion_limit': 25, 'configurable': {}}


수정한결과:
{'foo': 'bar'}


In [16]:
output

{'foo': 'bar'}

-----
* End of Document *